In [11]:
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, losses, metrics
from tensorflow.keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
import re

from konlpy.tag import Okt

import matplotlib.pyplot as plt

from konlpy.tag import Komoran
komoran = Komoran()

In [2]:
base_dir = 'C:/Users/user/Desktop/9.21(월)/현대 자료'  # 경로

csv_file = '현대차 2020-9-21  11시 1분 22초 merging.csv'      # 파일명

In [4]:
csv_dir = os.path.join(base_dir, csv_file)

csv_dir

'C:/Users/user/Desktop/9.21(월)/현대 자료\\현대차 2020-9-21  11시 1분 22초 merging.csv'

In [5]:
df = pd.read_csv(csv_dir, engine='python', encoding='CP949')

x = df['title']
x

0                                현대차, WRC 5차전에서 2, 3위 석권
1                     현대차 "한국형 도심항공교통 추진팀 결성"…시험비행 준비 박차
2                                현대차, 납품대금 1조1087억 조기 지급
3                         현대차·인천공항·현대건설·KT, 도심에 ‘하늘길’ 연다
4                        [특징주] 현대차 3%대 상승…셀트리온 제치고 시총 6위
5                     현대차, 하반기 신차 출시·원가 개선 효과 본격화...목표가↑
6                          美·유럽 뻗어나가는 현대차 수소트럭…세계시장 '주목'
7                     현대차, WRC 터키대회서 2연속 '더블 포디움'… 종합 2위
8                       현대차·LG·KT·네이버 총출동…2.5조 미래도시 ‘전쟁’
9                                     하반기 ‘야심작’ 시동 건 현대차
10                   현대차·인천공항·현대건설·KT, 한국형 UAM 사업 협력 MOU
11                            [펀드와치]조용한 시장에…현대차 펀드 단독 질주
12                   현대차 “추석 전 협력사에 납품대금 1조1000억원 조기 지급”
13                                    하반기 ‘야심작’ 시동 건 현대차
14                                   현대차 수소전기트럭 中서 기술혁신상
15                       [특징주] 현대차 3%대 상승…셀트리온 제치고 시총 6위
16                              현대차 울산공장 30대 직원 코로나19 감염
17                    "현대차, 실적 

In [6]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
OOV_INDEX = 1

# 데이터 타입
#ENCODER_INPUT  = 0
#DECODER_INPUT  = 1
#DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수 
max_sequences = 100
max_words=100
# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

# 정규 표현식 필터
RE_FILTER = re.compile("=[.,!?\"':;~()]")

In [17]:
# 형태소분석 함수
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        sentence = re.sub(RE_FILTER, "", str(sentence))
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        sentence = " ".join(komoran.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

In [18]:
x_new = pos_tag(x)


sentences = []
sentences.extend(x_new)


words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, OOV]

#print(words) # words : test,trainset의 모든 단어들이 들어간 list

# 단어와 인덱스의 단어사전 생성
word_to_index = {word: index for index, word in enumerate(words)}  # 단어 : 인덱스값 (인코딩시 사용)  # 단어 : 인덱스값
index_to_word = {index: word for index, word in enumerate(words)}  # 인덱스값 : 단어 (디코딩시 사용)  # 인덱스값 : 단어

x_new

['현대차 , WRC 5 차전 에서 2 , 3 위 석권',
 '현대차 " 한국 형 도심 항공 교통 추진 팀 결성 " … 시험 비행 준비 박차',
 '현대차 , 납품 대금 1 조 1087 억 조기 지급',
 '현대차 · 인천 공항 · 현대건설 · KT , 도심 에 ‘ 하늘 길 ’ 열 ㄴ다',
 '[ 특징 주 ] 현대차 3 % 대 상승 … 셀트리온 제치 고 시 총 6 위',
 '현대차 , 하반기 신 차 출시 · 원가 개선 효과 본격 화 ... 목표 가 ↑',
 "美 · 유럽 뻗 어 나가 는 현대차 수소 트럭 … 세계시 장 ' 주목 '",
 "현대차 , WRC 터키 대회 서 2 연속 ' 더블 포디움'… 종합 2 위",
 '현대차 · LG · KT · 네이버 총출동 … 2 . 5 조 미래 도시 ‘ 전쟁 ’',
 '하반기 ‘ 야심작 ’ 시동 건 현대차',
 '현대차 · 인천 공항 · 현대건설 · KT , 한국 형 UAM 사업 협력 MOU',
 '[ 펀드 와 하 지 ] 조용 하 ㄴ 시장 에 … 현대차 펀드 단독 질주',
 '현대차 “ 추석 전 협력 사 에 납품 대금 1 조 1000 억 원 조기 지급 ”',
 '하반기 ‘ 야심작 ’ 시동 건 현대차',
 '현대차 수소 전기 트럭 中 서 기술 혁신 상',
 '[ 특징 주 ] 현대차 3 % 대 상승 … 셀트리온 제치 고 시 총 6 위',
 '현대차 울산 공장 30대 직원 코로나 19 감염',
 '" 현대차 , 실적 턴 어 라운드 지속 … 목표 가 21 만원 " - SK증권',
 '현대차 노사 , 올해 임금 협상 추석 전 타결 위하 ㄴ 마지막 교섭',
 "원 · 달러 환율 급락 에 외국인 컴백 ? … 삼성전자 · 현대차 ' 수혜 '",
 "현대차 ' 차 박 ' 플랫폼 ' 휠 핑 ' 운영 … 1 박 2 일 차량 대 이 어",
 "니콜 이 라 사기 설 에 현대차 ' 반짝 ' … 수소 차 투자 전망 은",
 '현대차 전시장 , 몰라보 게 달라지 ㄴ다 … 송파대로 에 첫 서 ㄴ',
 "[ 특징 주 ] 현대차 , 3 분기 실적 개선 기대

In [19]:
import operator

lst=[]
for i in range(len(x_new)):
    w=x_new[i]
    w=w.split(' ')
    for j in range(len(w)):
        lst.append(w[j])

d={}
ex_t='[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…...》'

for i in lst:
    if i in ex_t:
        continue
    if i not in d:
        d[i]=1
    else:
        d[i]+=1
        
d=sorted(d.items(), key = lambda item: item[1])
d2={}

for i in range(len(d)):
    d2[d[i][0]]=d[i][1]
    

for k,v in d2.items():
    print('{} : {}'.format(k,v))

포디움'… : 1
개국 : 1
석권 : 2
원가 : 2
뻗 : 2
나가 : 2
세계시 : 2
총출동 : 2
도시 : 2
전쟁 : 2
조용 : 2
질주 : 2
1000 : 2
30대 : 2
런칭…10월 : 2
달 : 2
싼타페와 : 2
빌리 : 2
행사 : 2
韓 : 2
가속도 : 2
인프라 : 2
에어택시 : 2
관리 : 2
부서 : 2
근무 : 2
수직 : 2
이착륙기 : 2
만들 : 2
증상 : 2
발현 : 2
iTN : 2
따르 : 2
상향 : 2
비용 : 2
축소 : 2
유상증자 : 2
주주 : 2
희석 : 2
↓ : 2
본사 : 2
사라지 : 2
관계없 : 2
유발 : 2
준우승 : 2
포디움' : 2
업무 : 2
모으 : 2
빛 : 2
송파 : 2
아서 : 2
체계 : 2
론칭...'싼타페'·'투싼' : 2
車泊 : 2
속보 : 2
국 : 2
활짝 : 2
아냐 : 2
실험 : 2
가져오 : 2
어디 : 2
든 : 2
런칭… : 2
손쉽 : 2
를 : 2
기로 : 2
MO : 2
7 : 2
근무자 : 2
족 : 2
잡 : 2
세우 : 2
장례 : 2
압박 : 2
수단 : 2
불과 : 2
항공사 : 2
모시 : 2
ㅂ니다 : 2
권 : 2
트랙터 : 2
편의 : 2
제고 : 2
박하 : 2
요 : 2
GRZ : 2
테크놀로지스에 : 2
휘청대 : 2
구성 : 2
向 : 2
더블포디움 : 2
싼타페-투싼 : 2
파손 : 2
동구 : 2
이웃 : 2
돕 : 2
생필품 : 2
나누 : 2
ㅁ : 2
층 : 2
매장 : 2
출시…투싼·싼타페 : 2
이틀 : 2
선서 : 2
소속 : 2
선수 : 2
코 : 2
로나 : 2
실물 : 2
연구개발 : 2
매출 : 2
대비 : 2
년래 : 2
산타페 : 2
대박 : 2
700 : 2
위풍 : 2
라는 : 2
찍 : 2
간다 : 2
스포 : 2
티 : 2
정규 : 2
직 : 2
채용 : 2
며 : 2
窓 : 2
다가오 : 2
시행 : 2
단계 : 2
연장 : 2
자리매김 : 2
연합군 : 2
혁명 : 2
itM : 2


In [20]:
res=pd.DataFrame(list(d2.items()),
                   columns=['단어', '빈도수'])
res.to_excel('현대차_텍스트마이닝_기사제목.xlsx')